## Sea Surface Temperature (SST) anomalies

Sea surface temperature anomalies measure the difference in temperature with respect to average conditions of the ocean. So, a positive difference means the surface ocean is warmer than normal and a negative difference means it's cooler than normal.  

In the graph below, redder colors indicate a positive difference and bluer colors indicate a negative difference. Does this make sense based on our discussion of how temperature is distributed around the Earth?

![SST anomaly](https://www.ospo.noaa.gov/data/sst/anomaly/2019/anomnight.1.3.2019.gif)

We are going to calculate sea surface temperature anomalies for the SODA dataset today.  SODA or Simple Ocean Data Assimilation combines experimental measurements of the ocean with modelled data to map out ocean conditions across the globe. The oceans are huge (almost 70% of the Earth's surface is covered by oceans) so it very difficult to get oceanographic measurements of every inch of it. Scientists instead use models based on their understanding of what influences the oceans to fill in the gaps in their measurements. They can also use models to go back in time to look at what oceans might have been like in the past.

### Pseudocode
Here are the steps we need to take:
1. import file
1. save the necessary variables
1. find the mean temperature
1. find SST anomaly

#### Step 1: Import file

```
change directory

import netcdf4 module

read in and save dataset as a variable called 'data'

look at the names of the variables
```

In [1]:
import os
from netCDF4 import Dataset
os.chdir('/Users/brownscholar/Dropbox/BridgeUP_ClimateCoders/Data')
data = Dataset('soda3.12.2_mn_ocean_reg_2017.nc')
data.variables.keys()

odict_keys(['time', 'xt_ocean', 'yt_ocean', 'xu_ocean', 'yu_ocean', 'st_ocean', 'sw_ocean', 'temp', 'salt', 'wt', 'ssh', 'mlt', 'mlp', 'mls', 'net_heating', 'prho', 'u', 'v', 'taux', 'tauy'])

In [2]:
import numpy as np

This is a description of what each of these variables are

In [3]:
for variable in data.variables:
    print(variable,': ',data.variables[variable].long_name)

time :  time
xt_ocean :  longitude
yt_ocean :  latitude
xu_ocean :  longitude
yu_ocean :  latitude
st_ocean :  tcell zstar depth
sw_ocean :  ucell zstar depth
temp :  Potential temperature
salt :  Practical Salinity
wt :  dia-surface velocity T-points
ssh :  effective sea level (eta_t + patm/(rho0*g)) on T cells
mlt :  mixed layer depth determined by temperature criteria
mlp :  Depth of potential density mixed layer
mls :  mixed layer depth determined by salinity criteria
net_heating :  surface ocean heat flux coming through coupler and mass transfer
prho :  potential density referenced to 0 dbar
u :  i-current
v :  j-current
taux :  i-directed wind stress forcing u-velocity
tauy :  j-directed wind stress forcing v-velocity


Use the code above to print out the shape of the variables. What do the dimensions of temp i.e temperature correspond to?

In [4]:
for variables in data.variables:
    print(variables,': ',data.variables[variables].shape)

time :  (12,)
xt_ocean :  (720,)
yt_ocean :  (330,)
xu_ocean :  (720,)
yu_ocean :  (330,)
st_ocean :  (50,)
sw_ocean :  (50,)
temp :  (12, 50, 330, 720)
salt :  (12, 50, 330, 720)
wt :  (12, 50, 330, 720)
ssh :  (12, 330, 720)
mlt :  (12, 330, 720)
mlp :  (12, 330, 720)
mls :  (12, 330, 720)
net_heating :  (12, 330, 720)
prho :  (12, 50, 330, 720)
u :  (12, 50, 330, 720)
v :  (12, 50, 330, 720)
taux :  (12, 330, 720)
tauy :  (12, 330, 720)


In [5]:
#data.variables

#### Step 2: Save the required variables

```
save the variables temperature depends on i.e time, st_ocean, yt_ocean, xt_ocean

save temperature i.e temp
```

In [6]:
# Save latitude as variable
lat = data.variables["yt_ocean"][:]
# Save longitude as variable
lon = data.variables["xt_ocean"][:]
#Save time as variable
time = data.variables["time"][:]
#Save depth as variable
depth = data.variables["st_ocean"][:]
# Save temperature as variable
temp = data.variables["temp"][:]

Print out the minimum and maximum of each variable. Do these values make sense?

In [7]:
import numpy as np

#You can use the min() and max() *method* to do this calculation.
A = np.array([1,2,3,4,5])
print('Example: ',A.min(),'\n')

print('lat min:',lat.min())
print('lat max:',lat.max())
print('lon min:',lon.min())
print('lon max:',lon.max())
print('time min:',time.min())
print('time max:',time.max())
print('depth min:',depth.min())
print('depth max:',depth.max())
print('temp min:',temp.min())
print('temp max:',temp.max())

Example:  1 

lat min: -74.75
lat max: 89.75
lon min: 0.25
lon max: 359.75
time min: 13530.041666666666
time max: 13860.041666666666
depth min: 5.033549785614014
depth max: 5395.02294921875
temp min: -1.9938582
temp max: 38.07777


Notice that time is measured in a different format as days from Jan 01, 1980. Sound familiar? We'll learn how to convert to a datetime format using the function in another session.

Draw out what the temp variable might look like.

#### Step 3: Find mean temperature 

*Hint*: Use https://docs.scipy.org/doc/numpy/reference/generated/numpy.ma.mean.html

First, we need to decide what depth we would consider to be surface ocean? Call this variable z.

In [8]:
z = 20

Then, let's do this step by step. 

1. Only obtain data for the first month. You should get a grid with a shape of (50, 330, 720) 

In [9]:
m = temp[0]

2. Identify the index for the first z m of the ocean
3. Save temperature for the top z m

In [10]:
#Tells where depth is less than 20
depth<z

masked_array(data=[ True,  True, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False],
             mask=False,
       fill_value=True)

In [11]:
temp_jan = temp[0,depth<20,:,:]

Average over all z m. You should get a grid that is 330 by 720.

In [12]:
temp_jan_mean = temp_jan.mean(axis = 0)
#Or u can do this and get the same result:
temp_jan.mean(axis = 0).mean()

14.015762457458468

Now, find the mean for the entire grid.

In [13]:
temp_jan_mean.shape

(330, 720)

#### But....

You can do this step all at once by just applying the mean function to your array of the first month down to a depth of z m. I just wanted to make sure you got the steps its taking.

In [14]:
print(temp_jan_mean.mean())

14.015762457458468


Now find the mean for all 12 months to a depth of z m.

In [15]:
temp_all = temp[:,depth<20,:,:]
print(temp_all.mean())

14.017760987122497


#### Step 3: Find SST anomaly

Remember:

SST anomaly = SST - average SST

In [16]:
temp[:,depth<20,:,:] - temp_all.mean()

masked_array(
  data=[[[[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [-15.731119155883789, -15.731095314025879,
           -15.731050491333008, ..., -15.731263160705566,
           -15.731220245361328, -15.731162071228027],
          [-15.732431411743164, -15.732502937316895,
           -15.732555389404297, ..., -15.732232093811035,
           -15.732311248779297, -15.73237419128418],
          [-15.737030982971191, -15.737079620361328,
           -15.737127304077148, ..., -15.736905097961426,
           -15.736946105957031, -15.736985206604004]],

         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [-15.73108196258545, -15.73105239868164, -15.73108959197998,
           ..., -15.731328010559082, -15.731200218200684,
           -15.731139183044434],
          [-15.731245994567871, -15.731307983398438,

## Challenge

Repeat these steps for sea surface salinity data ('salt')